In [2]:
import nltk
from nltk.stem import WordNetLemmatizer
import pickle
import numpy as np
import json
import random
from nltk.stem import WordNetLemmatizer
from keras.models import load_model
from tensorflow.keras.preprocessing.sequence import pad_sequences

In [3]:
lemmatizer = WordNetLemmatizer()
model = load_model('../models/chatbot_model.h5')
intents = json.loads(open('../dataset/convo.json').read())
words = pickle.load(open('../obj2/words.pkl','rb'))
classes = pickle.load(open('../obj2/classes.pkl','rb'))

In [10]:
def clean_up_sentence(sentence):
    # tokenize the pattern - split words into array
    sentence_words = nltk.word_tokenize(sentence)
    # stem each word - create short form for word
    sentence_words = [lemmatizer.lemmatize(word.lower()) for word in sentence_words]
    return sentence_words
# return bag of words array: 0 or 1 for each word in the bag that exists in the sentence

In [11]:
def bow(sentence, words, show_details=True):
    # tokenize the pattern
    sentence_words = clean_up_sentence(sentence)
    # bag of words - matrix of N words, vocabulary matrix
    bag = [0]*len(words) 
    for s in sentence_words:
        for i,w in enumerate(words):
            if w == s: 
                # assign 1 if current word is in the vocabulary position
                bag[i] = 1
                if show_details:
                    print ("found in bag: %s" % w)
    return(np.array(bag))

In [12]:
def predict_class(sentence, model):
    # filter out predictions below a threshold
    p = bow(sentence, words,show_details=False)
    res = model.predict(np.array([p]))[0]
    ERROR_THRESHOLD = 0.25
    results = [[i,r] for i,r in enumerate(res) if r>ERROR_THRESHOLD]
    # sort by strength of probability
    results.sort(key=lambda x: x[1], reverse=True)
    return_list = []
    for r in results:
        return_list.append({"intent": classes[r[0]], "probability": str(r[1])})
    return return_list

In [13]:
def getResponse(ints, intents_json):
    tag = ints[0]['intent']
    list_of_intents = intents_json['intents']
    for i in list_of_intents:
        if(i['tag']== tag):
            result = random.choice(i['responses'])
            break
    return result

def chatbot_response(text):
    ints = predict_class(text, model)
    res = getResponse(ints, intents)
    return res

In [15]:
def main():
    print("Start chatting with the bot (type 'quit' to stop)!")
    while True:
        message = input("You: ")
        if message.lower() == "quit":
            print("Goodbye!")
            break
        response = chatbot_response(message)
        print(f"Bot: {response}")

In [16]:
if __name__ == "__main__":
    main()

Start chatting with the bot (type 'quit' to stop)!
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 100ms/step
Bot: Ciao! What's on your mind?
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 26ms/step
Bot: That's fantastic! Anything special happening today?
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 24ms/step
Bot: I'm here to listen without judgment. Please share more.
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 25ms/step
Bot: It's important to talk about this. Please continue.
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 28ms/step
Bot: Let's delve deeper into this. Please share more.


KeyboardInterrupt: Interrupted by user